# Multi-Qubit Gates - Code Laboratory

**Section 1: Quantum Operations | Part 2 of 3**

> 📖 **For concepts, analogies, and theory**: See [README.md](README.md)
> 🔬 **This notebook**: Executable code demonstrations and API exploration

---

## 🚀 Quick API Reference

| Gate | Qiskit Method | Parameters | Effect |
|------|---------------|------------|--------|
| **CNOT** | `qc.cx(ctrl, tgt)` | ctrl, tgt: int | Flip target if ctrl=\|1⟩ |
| **CZ** | `qc.cz(q1, q2)` | q1, q2: int | Phase flip if both=\|1⟩ (symmetric!) |
| **SWAP** | `qc.swap(q1, q2)` | q1, q2: int | Exchange qubits |
| **Toffoli** | `qc.ccx(c1, c2, tgt)` | c1, c2, tgt: int | Flip if both controls=\|1⟩ |

---

## 📋 Notebook Contents

1. **Setup** - Imports and helper functions
2. **CNOT Gate** - Entanglement creator
3. **Bell States** - All 4 maximally entangled states
4. **CZ Gate** - Symmetric controlled-Z
5. **SWAP Gate** - Qubit exchange
6. **Toffoli Gate** - Double-controlled NOT
7. **Trap Demonstrations** - Common mistakes in code
8. **Code Challenges** - Test your understanding

---

## Setup

---

## ⚠️ CRITICAL: Controlled Gates API Pattern

All controlled gates follow: **control qubit(s) FIRST, target LAST**

| Gate | Syntax | Control | Target |
|------|--------|---------|--------|
| **CX/CNOT** | `qc.cx(ctrl, tgt)` | ctrl | tgt |
| **CY** | `qc.cy(ctrl, tgt)` | ctrl | tgt |
| **CZ** | `qc.cz(q1, q2)` | either | either ✅ |
| **CCX/Toffoli** | `qc.ccx(c1, c2, tgt)` | c1, c2 | tgt |
| **CSWAP/Fredkin** | `qc.cswap(ctrl, t1, t2)` | ctrl | t1, t2 |

> ⚠️ **TRAP**: CX direction matters! CZ is symmetric (exception)!

In [ ]:
# Controlled Gates API Pattern: Control FIRST, Target LAST
from qiskit.quantum_info import Operator

print("Controlled Gates Parameter Order")
print("=" * 60)

# All controlled gates: control(s) first, target last
print("\n📋 Standard Pattern: qc.cX(control, target)")

# CX - NOT symmetric
qc_cx_01 = QuantumCircuit(2)
qc_cx_01.cx(0, 1)  # q0 controls, q1 is target

qc_cx_10 = QuantumCircuit(2)
qc_cx_10.cx(1, 0)  # q1 controls, q0 is target

print(f"\nCX(0,1) == CX(1,0): {Operator(qc_cx_01).equiv(Operator(qc_cx_10))} ❌")
print("  → CNOT is NOT symmetric! Direction matters!")

# CZ - IS symmetric (the exception!)
qc_cz_01 = QuantumCircuit(2)
qc_cz_01.cz(0, 1)

qc_cz_10 = QuantumCircuit(2)
qc_cz_10.cz(1, 0)

print(f"\nCZ(0,1) == CZ(1,0): {Operator(qc_cz_01).equiv(Operator(qc_cz_10))} ✅")
print("  → CZ IS symmetric! Either qubit can be 'control'")

# CCX - Two controls, one target
qc_ccx = QuantumCircuit(3)
qc_ccx.ccx(0, 1, 2)  # q0, q1 control; q2 is target
print(f"\nCCX(0, 1, 2): controls are q0, q1; target is q2")

# CSWAP - One control, two swap targets
qc_cswap = QuantumCircuit(3)
qc_cswap.cswap(0, 1, 2)  # q0 controls; q1,q2 swap
print(f"CSWAP(0, 1, 2): q0 controls; q1 and q2 swap if q0=|1⟩")

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_bloch_multivector, plot_state_city
import matplotlib.pyplot as plt

%matplotlib inline

# Helper: verify 2-qubit state
def verify_state(qc, expected, name="State"):
    """Verify circuit produces expected state vector."""
    actual = Statevector(qc).data
    match = np.allclose(actual, expected, atol=1e-10)
    status = "✅ PASS" if match else "❌ FAIL"
    print(f"{name}: {status}")
    return match

print("✅ Setup complete!")

---

## 2. CNOT Gate (CX)

### API Reference
```python
qc.cx(control, target)   # Control FIRST, target SECOND
qc.cnot(control, target) # Alternative name
```

> 📖 Theory: See README.md for truth table and matrix

In [ ]:
# CNOT Gate - Circuit, Matrix, and Truth Table
qc_cnot = QuantumCircuit(2)
qc_cnot.cx(0, 1)
display(qc_cnot.draw(output='mpl'))

print("Matrix:", Operator(qc_cnot).data.real.astype(int))

# Truth table verification
print("\nTruth Table: CX(0,1)")
for bits in ['00', '01', '10', '11']:
    qc = QuantumCircuit(2)
    if bits[0] == '1': qc.x(0)
    if bits[1] == '1': qc.x(1)
    qc.cx(0, 1)
    out = format(np.argmax(np.abs(Statevector(qc).data)), '02b')
    marker = " ← FLIP" if bits[0] == '1' else ""
    print(f"|{bits}⟩ → |{out}⟩{marker}")

In [ ]:
# Creating Entanglement: H + CNOT → Bell State
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)

state = Statevector(qc_bell)
print("H + CNOT creates Bell state |Φ+⟩:")
display(qc_bell.draw(output='mpl'))

print(f"\nState vector: {np.round(state.data, 4)}")
print("= (|00⟩ + |11⟩)/√2")

plot_state_city(state)

In [ ]:
# ⚠️ TRAP: CX(0,1) ≠ CX(1,0)
# Start with |10⟩

# CX(0,1): q0 controls q1
qc_01 = QuantumCircuit(2)
qc_01.x(0)      # |10⟩
qc_01.cx(0, 1)  # Control=1, so flip target

state_01 = Statevector(qc_01).data
out_01 = format(np.argmax(np.abs(state_01)), '02b')

# CX(1,0): q1 controls q0
qc_10 = QuantumCircuit(2)
qc_10.x(0)      # |10⟩
qc_10.cx(1, 0)  # Control=0, so NO flip

state_10 = Statevector(qc_10).data
out_10 = format(np.argmax(np.abs(state_10)), '02b')

print("⚠️ CNOT Direction Trap")
print("=" * 40)
print(f"Input: |10⟩")
print(f"CX(0,1): |10⟩ → |{out_01}⟩  (control=1, FLIPS)")
print(f"CX(1,0): |10⟩ → |{out_10}⟩  (control=0, NO flip)")
print(f"\n❌ CX(0,1) ≠ CX(1,0)!")

---

## 3. CZ Gate (Controlled-Z)

### API Reference
```python
qc.cz(qubit1, qubit2)  # Symmetric! Order doesn't matter
```

> 📖 Theory: CZ adds -1 phase to |11⟩ only

In [ ]:
# CZ Gate - Symmetric! CZ(0,1) == CZ(1,0)
qc_cz = QuantumCircuit(2)
qc_cz.cz(0, 1)
display(qc_cz.draw(output='mpl'))

print("Matrix (diagonal - only |11⟩ gets -1):")
print(Operator(qc_cz).data.real.astype(int))

# Verify symmetry
qc1, qc2 = QuantumCircuit(2), QuantumCircuit(2)
qc1.cz(0, 1); qc2.cz(1, 0)
print(f"\nCZ(0,1) == CZ(1,0): {Operator(qc1).equiv(Operator(qc2))} ✅")

---

## 4. SWAP Gate

### API Reference
```python
qc.swap(qubit1, qubit2)  # Exchange qubits
```

> 📖 Theory: SWAP = 3 CNOTs

In [ ]:
# SWAP Gate - Basic Usage & Verification
qc_swap = QuantumCircuit(2)
qc_swap.swap(0, 1)
display(qc_swap.draw(output='mpl'))

# Verify: |01⟩ → |10⟩
qc_test = QuantumCircuit(2)
qc_test.x(1)  # Start |01⟩
qc_test.swap(0, 1)
out = format(np.argmax(np.abs(Statevector(qc_test).data)), '02b')
print(f"SWAP |01⟩ → |{out}⟩ ✅")

In [ ]:
# SWAP Decomposition: 3 CNOTs
qc_decomp = QuantumCircuit(2)
qc_decomp.cx(0, 1)
qc_decomp.cx(1, 0)
qc_decomp.cx(0, 1)

print("SWAP = 3 CNOTs:")
display(qc_decomp.draw(output='mpl'))

# Verify equivalence
swap_m = Operator(qc_swap).data
decomp_m = Operator(qc_decomp).data
print(f"\nSWAP = CX·CX·CX: {np.allclose(swap_m, decomp_m)} ✅")

---

## 5. Toffoli Gate (CCX)

### API Reference
```python
qc.ccx(control1, control2, target)  # Double-controlled NOT
qc.toffoli(control1, control2, target)  # Alternative name
```

> 📖 Theory: Implements AND gate in quantum circuits

In [ ]:
# Toffoli Gate - Double-controlled NOT (implements AND)
qc_ccx = QuantumCircuit(3)
qc_ccx.ccx(0, 1, 2)
display(qc_ccx.draw(output='mpl'))

# Truth table (flips only when BOTH controls = 1)
print("Truth Table: CCX(0,1,2)")
for bits in ['000', '001', '010', '011', '100', '101', '110', '111']:
    qc = QuantumCircuit(3)
    for i, b in enumerate(bits):
        if b == '1': qc.x(i)
    qc.ccx(0, 1, 2)
    out = format(np.argmax(np.abs(Statevector(qc).data)), '03b')
    marker = " ← AND gate" if bits[:2] == '11' else ""
    print(f"|{bits}⟩ → |{out}⟩{marker}")

---

## 6. Bell States (All 4)

### Recipes
| State | Formula | Circuit |
|-------|---------|---------|
| \|Φ⁺⟩ | (00+11)/√2 | H(0), CX(0,1) |
| \|Φ⁻⟩ | (00-11)/√2 | X(0), H(0), CX(0,1) |
| \|Ψ⁺⟩ | (01+10)/√2 | H(0), CX(0,1), X(1) |
| \|Ψ⁻⟩ | (01-10)/√2 | X(0), H(0), CX(0,1), X(1) |

> 📖 Memory: Φ=same bits (00,11), Ψ=different bits (01,10)

In [ ]:
# All 4 Bell States with helper function
def make_bell(name):
    """Create Bell state: 'Φ+', 'Φ-', 'Ψ+', 'Ψ-'"""
    qc = QuantumCircuit(2)
    qc.h(0)
    if '-' in name: qc.z(0)   # Negative phase
    if 'Ψ' in name: qc.x(1)   # Different bits
    qc.cx(0, 1)
    return qc

# Generate and verify all 4
print("Bell States: |Φ⟩=same bits (00,11), |Ψ⟩=diff bits (01,10)")
print("=" * 60)
for name in ['Φ+', 'Φ-', 'Ψ+', 'Ψ-']:
    qc = make_bell(name)
    sv = Statevector(qc)
    print(f"|{name}⟩: {np.round(sv.data, 3)}")

# Alternative recipe trap: X(0),H(0),CX also makes Φ-
qc_alt = QuantumCircuit(2)
qc_alt.x(0); qc_alt.h(0); qc_alt.cx(0, 1)
print(f"\nAlt recipe X,H,CX = Φ-: {np.allclose(Statevector(qc_alt).data, Statevector(make_bell('Φ-')).data)}")

In [ ]:
# Bell State Mnemonic: Φ = same bits (00,11), Ψ = different bits (01,10)
print("Bell State Mnemonic")
print("=" * 60)

# Verify Φ states have same bits (00, 11)
print("\n|Φ⟩ states contain |00⟩ and |11⟩ (SAME bits):")
for name, sign in [('Φ+', '+'), ('Φ-', '-')]:
    qc = QuantumCircuit(2)
    qc.h(0)
    if sign == '-': qc.z(0)
    qc.cx(0, 1)
    sv = Statevector(qc)
    nonzero = [(f"|{i:02b}⟩", f"{sv.data[i]:.3f}") for i in range(4) if abs(sv.data[i]) > 0.01]
    print(f"  |{name}⟩: {nonzero}")

# Verify Ψ states have different bits (01, 10)
print("\n|Ψ⟩ states contain |01⟩ and |10⟩ (DIFFERENT bits):")
for name, sign in [('Ψ+', '+'), ('Ψ-', '-')]:
    qc = QuantumCircuit(2)
    qc.h(0)
    if sign == '-': qc.z(0)
    qc.x(1)
    qc.cx(0, 1)
    sv = Statevector(qc)
    nonzero = [(f"|{i:02b}⟩", f"{sv.data[i]:.3f}") for i in range(4) if abs(sv.data[i]) > 0.01]
    print(f"  |{name}⟩: {nonzero}")

print("\n📋 MNEMONIC: 'Phi = Phifty-Phifty same' (Φ → same bits)")
print("            'Psi = Psydifferent' (Ψ → different bits)")
print("   + = positive superposition, - = negative superposition")

---

## 7. Additional Gates & Patterns

In [ ]:
# Phase Kickback Demo (critical for Deutsch-Jozsa, Grover)
qc = QuantumCircuit(2)
qc.h(0)        # Control in |+⟩
qc.x(1); qc.h(1)  # Target in |-⟩
qc.cx(0, 1)    # Phase kicks back to control!
qc.h(1)        # Return target

print("Phase Kickback: phase transfers from target to control")
display(qc.draw(output='mpl'))
print(f"Final state: {Statevector(qc).data}")

In [ ]:
# GHZ States: H + CNOT chain (generalizes Bell states to n qubits)
def create_ghz(n):
    """Create n-qubit GHZ state: (|00...0⟩ + |11...1⟩)/√2"""
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(n-1):
        qc.cx(i, i+1)
    return qc

# Demo: 3 and 4 qubit GHZ
for n in [3, 4]:
    qc = create_ghz(n)
    sv = Statevector(qc)
    print(f"GHZ-{n}: {sv.data}")
    display(qc.draw(output='mpl'))

In [ ]:
# Fredkin Gate (CSWAP) - swaps targets IF control=|1⟩
qc = QuantumCircuit(3)
qc.x(0); qc.x(1)  # |1,1,0⟩
qc.cswap(0, 1, 2)
print("Fredkin: qc.cswap(control, target1, target2)")
print(f"|1,1,0⟩ → |1,0,1⟩ (swapped): {Statevector(qc).data}")
display(qc.draw(output='mpl'))

In [ ]:
# ⚠️ TRAP DEMONSTRATIONS (run to see differences!)

# TRAP 1: CNOT direction matters - CX(0,1) ≠ CX(1,0)
print("=" * 60)
print("TRAP 1: CNOT Direction")
qc1, qc2 = QuantumCircuit(2), QuantumCircuit(2)
qc1.h(0); qc1.cx(0, 1)  # Correct for Bell
qc2.h(0); qc2.cx(1, 0)  # Wrong!
print(f"H + CX(0,1): {Statevector(qc1).data}")
print(f"H + CX(1,0): {Statevector(qc2).data}")
print("❌ Different! Direction matters for CNOT\n")

# TRAP 2: CZ IS symmetric - CZ(0,1) == CZ(1,0)
print("TRAP 2: CZ Symmetry")
qc3, qc4 = QuantumCircuit(2), QuantumCircuit(2)
qc3.h([0,1]); qc3.cz(0, 1)
qc4.h([0,1]); qc4.cz(1, 0)
print(f"CZ(0,1) == CZ(1,0): {np.allclose(Statevector(qc3).data, Statevector(qc4).data)}")
print("✅ Same! CZ is symmetric (unlike CNOT)\n")

# TRAP 3: SWAP needs 3 CNOTs (not 2!)
print("TRAP 3: SWAP = 3 CNOTs")
qc_3cx = QuantumCircuit(2)
qc_3cx.cx(0,1); qc_3cx.cx(1,0); qc_3cx.cx(0,1)
qc_swap = QuantumCircuit(2)
qc_swap.swap(0, 1)
print(f"3 CNOTs == SWAP: {Operator(qc_3cx).equiv(Operator(qc_swap))}")
print("✅ SWAP = CX·CX·CX (3, not 2!)")

---

## 🎯 Master Trap Demonstrations

All critical Section 1 traps in one place for quick review:

In [ ]:
# 🎯 SECTION 1 MASTER TRAP QUICK REFERENCE
print("=" * 70)
print("🎯 SECTION 1: QUANTUM OPERATIONS - MASTER TRAP LIST")
print("=" * 70)

# Trap 1: Z|0⟩ = |0⟩ (unchanged!)
qc = QuantumCircuit(1)
qc.z(0)
sv = Statevector(qc)
print(f"\n1️⃣ Z|0⟩ = {sv.data} (UNCHANGED! Z only affects |1⟩)")

# Trap 2: CX direction matters
print(f"\n2️⃣ CX(0,1) ≠ CX(1,0) - Direction matters for CNOT!")

# Trap 3: CZ IS symmetric
print(f"3️⃣ CZ(0,1) = CZ(1,0) - CZ is symmetric (exception!)")

# Trap 4: SWAP = 3 CNOTs (not 2!)
print(f"4️⃣ SWAP = 3 CNOTs (CX·CX·CX), NOT 2!")

# Trap 5: T² = S, T⁴ = Z (NOT T² = Z!)
print(f"5️⃣ T² = S, T⁴ = Z (NOT T² = Z!)")

# Trap 6: HXH = Z, HZH = X
print(f"6️⃣ HXH = Z, HZH = X (Hadamard conjugation)")

# Trap 7: Pauli string ordering
print(f"7️⃣ Pauli('XYZ') = X on q2, Y on q1, Z on q0 (RIGHT-TO-LEFT!)")

# Trap 8: barrier() has NO quantum effect
print(f"8️⃣ barrier() is visual only - NO quantum effect!")

# Trap 9: initialize() is expensive
print(f"9️⃣ initialize() adds many gates - use gates for simple states!")

print("\n" + "=" * 70)
print("📋 MEMORIZE: Direction, Phase, Order - the 3 trap categories!")
print("=" * 70)

---

## 8. Code Challenges

> 📖 See README.md for concepts, analogies, mnemonics

In [ ]:
# CHALLENGE 1: Create 5-qubit GHZ state
# Your code here:
# qc = create_ghz(5)  # Use the helper function above

# CHALLENGE 2: Prove SWAP = 3 CNOTs (verify with Operator.equiv)
# Your code here:

# CHALLENGE 3: Build a Bell state using only CZ (not CX)
# Hint: H converts CZ to CX behavior
# Your code here:

# ─────────────────────────────────────────────────────────
# SOLUTIONS (run after trying yourself!)
print("Solutions:")

# 1. GHZ-5
qc = create_ghz(5)
print(f"GHZ-5 amplitudes: {len([a for a in Statevector(qc).data if abs(a) > 0.01])} non-zero")

# 2. SWAP = 3 CNOTs
qc_3cx = QuantumCircuit(2)
qc_3cx.cx(0,1); qc_3cx.cx(1,0); qc_3cx.cx(0,1)
qc_swap = QuantumCircuit(2); qc_swap.swap(0,1)
print(f"SWAP = 3 CNOTs: {Operator(qc_3cx).equiv(Operator(qc_swap))}")

# 3. Bell with CZ: H-H-CZ-H = H-CX (Bell state)
qc_cz_bell = QuantumCircuit(2)
qc_cz_bell.h([0,1]); qc_cz_bell.cz(0,1); qc_cz_bell.h(1)
print(f"CZ-based Bell: {np.round(Statevector(qc_cz_bell).data, 3)}")